### Some useful references:
1. **[Training]**: https://github.com/LIHANG-HONG/birdclef2023-2nd-place-solution
2. **[Inference]**: https://www.kaggle.com/code/kadircandrisolu/efficientnet-b0-pytorch-inference-birdclef-25

This model backbone is seresnext26t_32x4d

In [146]:
import os
import gc
import warnings
import logging
import time
import math
import cv2
from pathlib import Path
import joblib

import numpy as np
import pandas as pd
import librosa
import soundfile as sf
from soundfile import SoundFile 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import autocast, GradScaler
import timm
from tqdm.auto import tqdm
from glob import glob
import torchaudio
import random
import itertools
from typing import Union

import concurrent.futures

warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.ERROR)

In [147]:
class CFG:
    def __init__(self, mode="inference"):
        self.seed = 42
        self.print_freq = 100
        self.num_workers = 4
        self.stage = "train_bce"
        self.mode = mode  # "train" or "inference"

        # ===== Path Settings =====
        self.train_datadir = "/kaggle/input/birdclef-2025/train_audio"
        self.train_csv = "/kaggle/input/birdclef-2025/train.csv"
        self.test_soundscapes = "../data/raw/test_soundscapes_small/"
        self.submission_csv = "../data/raw/sample_submission.csv"
        self.taxonomy_csv = "../data/raw/taxonomy.csv"
        self.model_files = ["../models/sedmodel_0857/model_fold0.pth"]

        # ===== Model Settings =====
        self.model_name = "seresnext26t_32x4d"
        self.pretrained = False
        self.in_channels = 1

        # ===== Audio Settings =====
        self.SR = 32000
        self.target_duration = 5
        self.train_duration = 10
        
        cfg.n_mels = 128
        cfg.n_fft = 2048
        cfg.hop_length = 512
        cfg.f_min = 20
        cfg.f_max = 16000
        cfg.normal = 80  # or 255
        cfg.infer_duration = 5
        cfg.duration_train = 10

        # ===== Device =====
        self.device = "cpu"

cfg = CFG()

In [148]:
print(f"Using device: {cfg.device}")
print(f"Loading taxonomy data...")
taxonomy_df = pd.read_csv(cfg.taxonomy_csv)
species_ids = taxonomy_df['primary_label'].tolist()
num_classes = len(species_ids)
print(f"Number of classes: {num_classes}")

Using device: cpu
Loading taxonomy data...
Number of classes: 206


In [149]:
def set_seed(seed=42):
    """
    Set seed for reproducibility
    """
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(cfg.seed)


In [150]:
class AttBlockV2(nn.Module):
    def __init__(self, in_features: int, out_features: int, activation="linear"):
        super().__init__()

        self.activation = activation
        self.att = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True,
        )
        self.cla = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True,
        )

        self.init_weights()

    def init_weights(self):
        init_layer(self.att)
        init_layer(self.cla)

    def forward(self, x):
        # x: (n_samples, n_in, n_time)
        norm_att = torch.softmax(torch.tanh(self.att(x)), dim=-1)
        cla = self.nonlinear_transform(self.cla(x))
        x = torch.sum(norm_att * cla, dim=2)
        return x, norm_att, cla

    def nonlinear_transform(self, x):
        if self.activation == "linear":
            return x
        elif self.activation == "sigmoid":
            return torch.sigmoid(x)


def init_layer(layer):
    nn.init.xavier_uniform_(layer.weight)

    if hasattr(layer, "bias"):
        if layer.bias is not None:
            layer.bias.data.fill_(0.0)

def init_bn(bn):
    bn.bias.data.fill_(0.0)
    bn.weight.data.fill_(1.0)

In [151]:

class BirdCLEFModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        
        taxonomy_df = pd.read_csv(cfg.taxonomy_csv)
        self.num_classes = len(taxonomy_df)

        self.bn0 = nn.BatchNorm2d(cfg.n_mels)
        
        self.backbone = timm.create_model(
            cfg.model_name,
            pretrained=False,
            in_chans=cfg.in_channels,
            drop_rate=0.2,
            drop_path_rate=0.2,
        )

        layers = list(self.backbone.children())[:-2]
        self.encoder = nn.Sequential(*layers)
        
        if "efficientnet" in cfg.model_name:
            backbone_out = self.backbone.classifier.in_features
        elif "eca" in cfg.model_name:
            backbone_out = self.backbone.head.fc.in_features
        elif "res" in cfg.model_name:
            backbone_out = self.backbone.fc.in_features
        else:
            backbone_out = self.backbone.num_features
            
        self.fc1 = nn.Linear(backbone_out, backbone_out, bias=True)
        self.att_block = AttBlockV2(backbone_out, self.num_classes, activation="sigmoid")

        self.melspec_transform = torchaudio.transforms.MelSpectrogram(
            sample_rate=cfg.SR,
            hop_length=cfg.hop_length,
            n_mels=cfg.n_mels,
            f_min=cfg.f_min,
            f_max=cfg.f_max,
            n_fft=cfg.n_fft,
            pad_mode="constant",
            norm="slaney",
            onesided=True,
            mel_scale="htk",
        )
        if cfg.device == "cuda":
            self.melspec_transform = self.melspec_transform.cuda()
        else:
            self.melspec_transform = self.melspec_transform.cpu()

        self.db_transform = torchaudio.transforms.AmplitudeToDB(
            stype="power", top_db=80
        )

    def extract_feature(self, x):
        x = x.permute((0, 1, 3, 2))
        frames_num = x.shape[2]
        
        x = x.transpose(1, 3)
        x = self.bn0(x)
        x = x.transpose(1, 3)
        
        x = x.transpose(2, 3)
        x = self.encoder(x)
        
        x = torch.mean(x, dim=2)
        
        x1 = F.max_pool1d(x, kernel_size=3, stride=1, padding=1)
        x2 = F.avg_pool1d(x, kernel_size=3, stride=1, padding=1)
        x = x1 + x2
        
        x = F.dropout(x, p=0.5, training=self.training)
        x = x.transpose(1, 2)
        x = F.relu_(self.fc1(x))
        x = x.transpose(1, 2)
        x = F.dropout(x, p=0.5, training=self.training)
        return x, frames_num
        
    @torch.cuda.amp.autocast(enabled=False)
    def transform_to_spec(self, audio):
        audio = audio.float()
        spec = self.melspec_transform(audio)
        spec = self.db_transform(spec)

        if self.cfg.normal == 80:
            spec = (spec + 80) / 80
        elif self.cfg.normal == 255:
            spec = spec / 255
        else:
            raise NotImplementedError
                
        if self.cfg.in_channels == 3:
            spec = image_delta(spec)
        
        return spec

    def forward(self, x):
        with torch.no_grad():
            x = self.transform_to_spec(x)

        x, frames_num = self.extract_feature(x)
        
        clipwise_output, norm_att, segmentwise_output = self.att_block(x)
        logit = torch.sum(norm_att * self.att_block.cla(x), dim=2)
        segmentwise_logit = self.att_block.cla(x).transpose(1, 2)
        segmentwise_output = segmentwise_output.transpose(1, 2)

        return torch.logit(clipwise_output)

    def infer(self, x, tta_delta=2):
        with torch.no_grad():
            x = self.transform_to_spec(x)
        x, _ = self.extract_feature(x)
        time_att = torch.tanh(self.att_block.att(x))
        feat_time = x.size(-1)

        start = (
            feat_time / 2 - feat_time * (self.cfg.infer_duration / self.cfg.duration_train) / 2
        )
        end = start + feat_time * (self.cfg.infer_duration / self.cfg.duration_train)
        start = int(start)
        end = int(end)
        pred = self.attention_infer(start, end, x, time_att)

        start_minus = max(0, start - tta_delta)
        end_minus = end - tta_delta
        pred_minus = self.attention_infer(start_minus, end_minus, x, time_att)

        start_plus = start + tta_delta
        end_plus = min(feat_time, end + tta_delta)
        pred_plus = self.attention_infer(start_plus, end_plus, x, time_att)

        pred = 0.5 * pred + 0.25 * pred_minus + 0.25 * pred_plus
        return pred
        
    def attention_infer(self, start, end, x, time_att):
        feat = x[:, :, start:end]
        framewise_pred = torch.sigmoid(self.att_block.cla(feat))
        framewise_pred_max = framewise_pred.max(dim=2)[0]
        return framewise_pred_max

In [152]:
def load_sample(path, cfg):
    audio, orig_sr = sf.read(path, dtype="float32")
    seconds = []
    audio_length = cfg.SR * cfg.target_duration
    step = audio_length
    for i in range(audio_length, len(audio) + step, step):
        start = max(0, i - audio_length)
        end = start + audio_length
        if end > len(audio):
            pass
        else:
            seconds.append(int(end/cfg.SR))

    audio = np.concatenate([audio,audio,audio])
    audios = []
    for i,second in enumerate(seconds):
        end_seconds = int(second)
        start_seconds = int(end_seconds - cfg.target_duration)
    
        end_index = int(cfg.SR * (end_seconds + (cfg.train_duration - cfg.target_duration) / 2) ) + len(audio) // 3
        start_index = int(cfg.SR * (start_seconds - (cfg.train_duration - cfg.target_duration) / 2) ) + len(audio) // 3
        end_pad = int(cfg.SR * (cfg.train_duration - cfg.target_duration) / 2) 
        start_pad = int(cfg.SR * (cfg.train_duration - cfg.target_duration) / 2) 
        y = audio[start_index:end_index].astype(np.float32)
        if i==0:
            y[:start_pad] = 0
        elif i==(len(seconds)-1):
            y[-end_pad:] = 0
        audios.append(y)

    return audios

def sigmoid(x):
    s = 1 / (1 + np.exp(-x))
    return s

In [153]:
def load_models(cfg, num_classes):
    """
    Load all model checkpoints from cfg.model_files and return initialized models
    """
    models = []
    model_files = cfg.model_files

    if not model_files:
        print(f"⚠️ No model files found under {cfg.model_path}!")
        return models

    print(f"🔍 Found {len(model_files)} model file(s).")

    for i, model_path in enumerate(model_files):
        try:
            print(f"📦 Loading model: {model_path}")
            checkpoint = torch.load(model_path, map_location=torch.device(cfg.device))

            # checkpoint["cfg"] は dict の場合を想定して再構築
            if isinstance(checkpoint.get("cfg"), dict):
                cfg_dict = checkpoint["cfg"]
                cfg_temp = CFG()
                for k, v in cfg_dict.items():
                    setattr(cfg_temp, k, v)
            else:
                cfg_temp = checkpoint["cfg"]  # すでにCFGならそのまま使う

            cfg_temp.device = cfg.device  # 推論環境にあわせて上書き
            cfg_temp.taxonomy_csv = cfg.taxonomy_csv 

            model = BirdCLEFModel(cfg_temp)
            model.load_state_dict(checkpoint["model_state_dict"])
            model = model.to(cfg.device)
            model.eval()
            model.zero_grad()
            model.half().float()

            models.append(model)

        except Exception as e:
            print(f"❌ Error loading model {model_path}: {e}")

    return models

def predict_on_spectrogram(audio_path, models, cfg, species_ids):
    """Process a single audio file and predict species presence for each 5-second segment"""
    audio_path = str(audio_path)
    predictions = []
    row_ids = []
    soundscape_id = Path(audio_path).stem

    print(f"Processing {soundscape_id}")
    audio_data = load_sample(audio_path, cfg)
    for segment_idx, audio_input in enumerate(audio_data):
        
        end_time_sec = (segment_idx + 1) * cfg.target_duration
        row_id = f"{soundscape_id}_{end_time_sec}"
        row_ids.append(row_id)
        
        mel_spec = torch.tensor(audio_input, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
        mel_spec = mel_spec.to(cfg.device)
        
        if len(models) == 1:
            with torch.no_grad():
                outputs = models[0].infer(mel_spec)
                final_preds = outputs.squeeze()
                # final_preds = torch.sigmoid(outputs).cpu().numpy().squeeze()

        else:
            segment_preds = []
            for model in models:
                with torch.no_grad():
                    outputs = model.infer(mel_spec)
                    probs = outputs.squeeze()
                    # probs = torch.sigmoid(outputs).cpu().numpy().squeeze()
                    segment_preds.append(probs)

            
            final_preds = np.mean(segment_preds, axis=0)
                
        predictions.append(final_preds)

    predictions = np.stack(predictions,axis=0)
    
    return row_ids, predictions

In [154]:
def run_inference(cfg, models, species_ids):
    """Run inference on all test soundscapes"""
    test_files = sorted(Path(cfg.test_soundscapes).glob('*.ogg'))[:10]
    
    print(f"Found {len(test_files)} test soundscapes")

    all_row_ids = []
    all_predictions = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        results = list(
            executor.map(
                predict_on_spectrogram,
                test_files,
                itertools.repeat(models),
                itertools.repeat(cfg),
                itertools.repeat(species_ids)
            )
        )

    for rids, preds in results:
        all_row_ids.extend(rids)
        all_predictions.extend(preds)
    
    return all_row_ids, all_predictions

def create_submission(row_ids, predictions, species_ids, cfg):
    """Create submission dataframe"""
    print("Creating submission dataframe...")

    submission_dict = {'row_id': row_ids}
    
    for i, species in enumerate(species_ids):
        submission_dict[species] = [pred[i] for pred in predictions]

    submission_df = pd.DataFrame(submission_dict)

    submission_df.set_index('row_id', inplace=True)

    sample_sub = pd.read_csv(cfg.submission_csv, index_col='row_id')

    missing_cols = set(sample_sub.columns) - set(submission_df.columns)
    if missing_cols:
        print(f"Warning: Missing {len(missing_cols)} species columns in submission")
        for col in missing_cols:
            submission_df[col] = 0.0

    submission_df = submission_df[sample_sub.columns]

    submission_df = submission_df.reset_index()
    
    return submission_df


def smooth_submission(submission_path):
        """
        Post-process the submission CSV by smoothing predictions to enforce temporal consistency.
        
        For each soundscape (grouped by the file name part of 'row_id'), each row's predictions
        are averaged with those of its neighbors using defined weights.
        
        :param submission_path: Path to the submission CSV file.
        """
        print("Smoothing submission predictions...")
        sub = pd.read_csv(submission_path)
        cols = sub.columns[1:]
        # Extract group names by splitting row_id on the last underscore
        groups = sub['row_id'].str.rsplit('_', n=1).str[0].values
        unique_groups = np.unique(groups)
        
        for group in unique_groups:
            # Get indices for the current group
            idx = np.where(groups == group)[0]
            sub_group = sub.iloc[idx].copy()
            predictions = sub_group[cols].values
            new_predictions = predictions.copy()
            
            if predictions.shape[0] > 1:
                # Smooth the predictions using neighboring segments
                new_predictions[0] = (predictions[0] * 0.8) + (predictions[1] * 0.2)
                new_predictions[-1] = (predictions[-1] * 0.8) + (predictions[-2] * 0.2)
                for i in range(1, predictions.shape[0]-1):
                    new_predictions[i] = (predictions[i-1] * 0.2) + (predictions[i] * 0.6) + (predictions[i+1] * 0.2)
            # Replace the smoothed values in the submission dataframe
            sub.iloc[idx, 1:] = new_predictions
        
        sub.to_csv(submission_path, index=False)
        print(f"Smoothed submission saved to {submission_path}")

In [155]:

start_time = time.time()
print("Starting BirdCLEF-2025 inference...")

models = load_models(cfg, num_classes)

if not models:
    print("No models found! Please check model paths.")
    raise RuntimeError("No models loaded for inference.")

print(f"Model usage: {'Single model' if len(models) == 1 else f'Ensemble of {len(models)} models'}")

row_ids, predictions = run_inference(cfg, models, species_ids)

submission_df = create_submission(row_ids, predictions, species_ids, cfg)

submission_path = 'submission.csv'
submission_df.to_csv(submission_path, index=False)
print(f"Submission saved to {submission_path}")

smooth_submission(submission_path)

end_time = time.time()
print(f"Inference completed in {(end_time - start_time)/60:.2f} minutes")

Starting BirdCLEF-2025 inference...
🔍 Found 1 model file(s).
📦 Loading model: ../models/sedmodel_0857/model_fold0.pth


Model usage: Single model
Found 4 test soundscapes
Processing H02_20230420_074000
Processing H02_20230420_112000
Processing H02_20230420_154500
Processing H02_20230502_080500
Creating submission dataframe...
Submission saved to submission.csv
Smoothing submission predictions...
Smoothed submission saved to submission.csv
Inference completed in 0.12 minutes


In [158]:
submission_df.head()

,row_id,1139490,1192948,1194042,126247,1346504,134933,135045,1462711,1462737,...,yebfly1,yebsee1,yecspi2,yectyr1,yehbla2,yehcar1,yelori1,yeofly1,yercac1,ywcpar
0,H02_20230420_074000_5,0.019034,0.014293,0.009067,0.003697,0.010040,0.009429,0.008441,0.013995,0.016443,...,0.007549,0.007677,0.010365,0.016833,0.007334,0.010245,0.009943,0.013151,0.020206,0.004325
1,H02_20230420_074000_10,0.005526,0.005460,0.003589,0.009103,0.005507,0.011483,0.009386,0.006968,0.010645,...,0.007237,0.005919,0.006159,0.010452,0.005051,0.012963,0.006981,0.013079,0.007854,0.008929
2,H02_20230420_074000_15,0.009316,0.016221,0.007963,0.011094,0.010349,0.008795,0.008718,0.010356,0.014014,...,0.009298,0.007677,0.009697,0.014609,0.006194,0.015556,0.010150,0.014794,0.012863,0.011697
3,H02_20230420_074000_20,0.010347,0.015713,0.004209,0.009343,0.012749,0.009912,0.008312,0.011245,0.014436,...,0.009790,0.007069,0.006696,0.016567,0.007894,0.015836,0.011767,0.014757,0.012819,0.010654
4,H02_20230420_074000_25,0.005793,0.005728,0.005349,0.011468,0.007462,0.011525,0.009423,0.006088,0.009496,...,0.008974,0.007244,0.006177,0.018225,0.007333,0.015601,0.011264,0.013051,0.012356,0.011748
